Install

In [2]:
pip install Unidecode sastrawi python-Levenshtein git+https://github.com/ariaghora/mpstemmer.git

  Cloning https://github.com/ariaghora/mpstemmer.git to c:\users\l580\appdata\local\temp\pip-req-build-da9dgt41
  Resolved https://github.com/ariaghora/mpstemmer.git to commit caedf8d9a9219e4277973b80bdbd3eecb14a7e06
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/ariaghora/mpstemmer.git 'C:\Users\L580\AppData\Local\Temp\pip-req-build-da9dgt41'
You should consider upgrading via the 'c:\Users\L580\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
pip install nltk

You should consider upgrading via the 'c:\Users\L580\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install -U scikit-learn

     ---------------------------------------- 8.2/8.2 MB 507.4 kB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\L580\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
# Data Prerocessing
import re
from unidecode import unidecode
import string
import pandas as pd
import itertools

# Toknize
import nltk
from nltk.tokenize import word_tokenize,MWETokenizer
from nltk.probability import FreqDist
nltk.download('punkt')
nltk.download('stopwords')

# Stopword 
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords

# Stemmer
from mpstemmer import MPStemmer

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\L580\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\L580\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load Data

In [6]:
df_train = pd.read_csv(r'D:\Data Science - Binar Academy Wave 2\PLATINUM\Challange\challenge_platinum\train_preprocess.tsv', sep='\t',names = ['data','label'],encoding='latin-1')
df_train

,data,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


Data Description

In [7]:
# Info Data
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    11000 non-null  object
 1   label   11000 non-null  object
dtypes: object(2)
memory usage: 172.0+ KB


In [8]:
# Summarizing the Missing Value
df_train.isnull().sum()

data     0
label    0
dtype: int64

In [9]:
# Count Label
jumlah_label = df_train['label'].value_counts()
jumlah_label

positive    6416
negative    3436
neutral     1148
Name: label, dtype: int64

Cleansing Data

In [10]:
# function to case_fold
def case_fold(text):
    #membuat semua komentar menjadi huruf kecil 
    text = text.lower()
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    #replace - with space
    text = text.replace('-',' ')
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove consecutive word
    text = re.sub(r'\b(\w+\s*)\1{1,}', '\\1', text)
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    text.replace("http://", " ").replace("https://", " ")
    #remove punctuation
    text = text.translate(str.maketrans("","",string.punctuation))
    #remove whitestrip
    text = text.strip()
    #remove multiple whitespace into single whitespace
    text = re.sub('\s+',' ',text)
    #remove single char
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    #remove url
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)

    return text
df_train['case_fold'] = df_train['data'].apply(case_fold)
df_train

,data,label,case_fold
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini dimiliki oleh pengusaha pabrik tahu...
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral,mohon ulama lurus dan k212 mbri hujjah partai ...
2,lokasi strategis di jalan sumatera bandung . t...,positive,lokasi strategis di jalan sumatera bandung tem...
3,betapa bahagia nya diri ini saat unboxing pake...,positive,betapa bahagia nya diri ini saat unboxing pake...
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative,duh jadi mahasiswa jangan sombong dong kasih k...
...,...,...,...
10995,tidak kecewa,positive,tidak kecewa
10996,enak rasa masakan nya apalagi kepiting yang me...,positive,enak rasa masakan nya apalagi kepiting yang me...
10997,hormati partai-partai yang telah berkoalisi,neutral,hormati partai yang telah berkoalisi
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative,pagi di tol pasteur sudah macet parah bikin je...


In [17]:
#TOKENIZE
def multiword_tokenize(text):
  mwe = open("D:\Data Science - Binar Academy Wave 2\PLATINUM\Challange\challenge_platinum\mwe.txt", "r",).read().split("\n")
  protected_tuples = [word_tokenize(word) for word in mwe]
  protected_tuples_underscore = ['_'.join(word) for word in protected_tuples]
  tokenizer = MWETokenizer(protected_tuples)
  # Tokenize the text.
  tokenized_text = tokenizer.tokenize(word_tokenize(text))
  # Replace the underscored protected words with the original MWE
  for i, token in enumerate(tokenized_text):
    if token in protected_tuples_underscore:
      tokenized_text[i] = mwe[protected_tuples_underscore.index(token)]
    return tokenized_text

df_train['tokenize'] = df_train['case_fold'].apply(multiword_tokenize)
df_train = df_train [['data','case_fold','tokenize','label']]
df_train

,data,case_fold,tokenize,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,warung ini dimiliki oleh pengusaha pabrik tahu...,"[warung, ini, dimiliki, oleh, pengusaha, pabri...",positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,mohon ulama lurus dan k212 mbri hujjah partai ...,"[mohon, ulama, lurus, dan, k212, mbri, hujjah,...",neutral
2,lokasi strategis di jalan sumatera bandung . t...,lokasi strategis di jalan sumatera bandung tem...,"[lokasi, strategis, di, jalan, sumatera, bandu...",positive
3,betapa bahagia nya diri ini saat unboxing pake...,betapa bahagia nya diri ini saat unboxing pake...,"[betapa, bahagia, nya, diri, ini, saat, unboxi...",positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,duh jadi mahasiswa jangan sombong dong kasih k...,"[duh, jadi, mahasiswa, jangan, sombong, dong, ...",negative
...,...,...,...,...
10995,tidak kecewa,tidak kecewa,[tidak kecewa],positive
10996,enak rasa masakan nya apalagi kepiting yang me...,enak rasa masakan nya apalagi kepiting yang me...,"[enak, rasa, masakan, nya, apalagi, kepiting, ...",positive
10997,hormati partai-partai yang telah berkoalisi,hormati partai yang telah berkoalisi,"[hormati, partai, yang, telah, berkoalisi]",neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",pagi di tol pasteur sudah macet parah bikin je...,"[pagi, di, tol, pasteur, sudah, macet, parah, ...",negative
